In [1]:
#List of Imports
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Reading the colormetrics data
file_path = '40_cmet.csv'
df = pd.read_csv(file_path,index_col=False)
colormetrics = df.transpose()
colormetrics

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Unnamed: 0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000
Hour,0.000000,16.651000,39.288000,55.938000,72.589000,89.239000,89.822000,106.473000,123.123000,139.774000,156.424000,173.074000,189.725000,206.376000
0,0.256983,0.209864,0.293912,0.303905,0.296560,0.334347,0.333849,0.318610,0.333395,0.349732,0.364341,0.378918,0.391092,0.407256
1,0.269332,0.222227,0.303880,0.311916,0.304116,0.339371,0.340847,0.330932,0.349482,0.369055,0.386788,0.407936,0.420402,0.439380
2,0.258631,0.231534,0.303071,0.309405,0.300489,0.334048,0.334885,0.319176,0.338196,0.355248,0.372146,0.391818,0.404084,0.427026
3,0.228366,0.217902,0.275222,0.282441,0.277069,0.309991,0.310814,0.304538,0.317458,0.334819,0.349001,0.368835,0.379074,0.399881
4,0.230972,0.219111,0.267014,0.274603,0.271163,0.293367,0.293142,0.293086,0.310535,0.328821,0.345514,0.364140,0.381183,0.402279
5,0.208008,0.203909,0.230322,0.236671,0.249347,0.262192,0.260512,0.264734,0.280511,0.299753,0.317690,0.333903,0.351507,0.371698
6,0.193132,0.191564,0.215978,0.230189,0.241724,0.270953,0.269188,0.284693,0.301556,0.317973,0.335274,0.349631,0.366501,0.380648
7,0.204074,0.204206,0.227459,0.246179,0.254420,0.277694,0.277784,0.278798,0.294167,0.313176,0.325310,0.343192,0.358734,0.375369


In [3]:
#Reading in features from the PASCAL data
f = pd.read_json('maestro_sample_log.json')
df = f.transpose()
def extract_details(row):
    return row['worklist'][1]['details']['drops']
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
df = df.apply(extract_details,axis=1)
flattened_series = df.apply(lambda x: flatten_dict(x[0]))

# Sort the DataFrame index numerically
df_sorted = flattened_series.reindex(sorted(flattened_series.index, key=lambda x: int(x[6:])))

# Flatten the dictionary of composition data into their own columns
flattened_df = pd.DataFrame(df_sorted.tolist(), index=df_sorted.index)

#Drop specific samples based on colormetrics file
samples_to_ignore = ['sample17','sample22','sample38','sample39','sample40','sample44','sample41','sample42','sample43']
samples_to_keep = ~flattened_df.index.isin(samples_to_ignore)
cleaned = flattened_df[samples_to_keep]
cleaned

,air_gap,blow_out,height,pre_mix,rate,reuse_tip,slow_retract,slow_travel,solution_molarity,solution_solutes,solution_solvent,solution_well_labware,solution_well_well,time,touch_tip,volume
sample0,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.45_Cl0.15,solvent,15mL_Tray1,B1,-5,True,40
sample1,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.45_Cl0.15,solvent,15mL_Tray1,B1,-5,True,40
sample2,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.45_Cl0.15,solvent,15mL_Tray1,B1,-5,True,40
sample3,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.525_Cl0.075,solvent,96wellplate,A6,-5,True,40
sample4,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.525_Cl0.075,solvent,96wellplate,A6,-5,True,40
sample5,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.525_Cl0.075,solvent,96wellplate,A6,-5,True,40
sample6,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.6,solvent,15mL_Tray1,C2,-5,True,40
sample7,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.6,solvent,15mL_Tray1,C2,-5,True,40
sample8,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.4_Br0.6,solvent,15mL_Tray1,C2,-5,True,40
sample9,True,True,2,"[3, 50]",80,False,True,True,1,FA0.78_Cs0.22_Pb_I2.55_Br0.3_Cl0.15,solvent,96wellplate,A1,-5,True,40


In [4]:
#Calculating degredation rate that will be predicted
hour = float(colormetrics[colormetrics.columns[-1]].iloc[1])
calculate = (colormetrics[colormetrics.columns[-1]]-colormetrics[0])/hour
degredation_rate = calculate.iloc[2:]
degredation_rate

0     0.000728
1     0.000824
2     0.000816
3     0.000831
4     0.000830
5     0.000793
6     0.000909
7     0.000830
8     0.000827
9     0.000736
10    0.000865
11    0.000448
12    0.000537
13    0.000696
14    0.000701
15    0.000630
16    0.001039
18    0.000759
19    0.000937
20    0.000981
21    0.000743
23    0.000532
24    0.000505
25    0.000463
26    0.001058
27    0.001529
28    0.001615
29    0.001027
30    0.000955
31    0.001125
32    0.001025
33    0.000606
34    0.000633
35    0.000974
36    0.001023
37    0.001284
dtype: float64

In [5]:
# Defining features(X) and target variable(y)
X = cleaned[['solution_solutes', 'solution_molarity', 'solution_well_well']]
y = degredation_rate

# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing the data based on data type
preprocessor = ColumnTransformer(
    transformers=[
        ('solutes', OneHotEncoder(), ['solution_solutes', 'solution_well_well']),
        ('numerical', StandardScaler(), ['solution_molarity'])
    ],
)
#list of types of regressors to try
regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Huber Regressor': HuberRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'SVR': SVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor()
}

for name, regressor in regressors.items():
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', regressor)
    ])

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} - MSE on the test set: {mse}")

Linear Regression - MSE on the test set: 5.743797647943956e-08
Ridge Regression - MSE on the test set: 6.833433960402303e-08
Lasso Regression - MSE on the test set: 1.4185907639474214e-07
ElasticNet - MSE on the test set: 1.4185907639474214e-07
Huber Regressor - MSE on the test set: 4.52855501274295e-08
Decision Tree Regressor - MSE on the test set: 5.743797647943957e-08
Random Forest Regressor - MSE on the test set: 6.922389122798139e-08
Gradient Boosting Regressor - MSE on the test set: 5.820765961562565e-08
SVR - MSE on the test set: 1.4088423900314387e-07
K-Nearest Neighbors Regressor - MSE on the test set: 1.2148194129635658e-07
